# Support Vector Machines (SVMs)
In this notebook, we will learn a linear and kernalised method of SVMs, which can be used for both regression and classification. To start with, we will focus on binary classification. We will use stochastic gradient descent (SGD) for the optimisation of the hinge loss.

We will work with the [Breast Cancer Wisconsin (Diagnostic) Data Set](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data), which you first need to download and then load in this notebook. If you faced difficulties downloading this data set from Kaggle, you should download the file directly from Blackboard. The data set contains various aspects of cell nuclei of breast screening images of patients with _(malignant)_ and without _(benign)_ breast cancer. Our goal is to build a classification model that can take these aspects of an unseen breast screening image, and classify it as either malignant or benign.

If you run this notebook locally on your machine, you will simply need to place the `csv` file in the same directory as this notebook.
If you run this notebook on Google Colab, you will need to use

  `from google.colab import files`

  `upload = files.upload()`

and then upload it from your local downloads directory.

In [ ]:
# necessary imports
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('./data.csv')

# print shape and last 10 rows
print(data.shape)
data.tail(10)

We can see that our data set has 569 samples and 33 columns. The column `id` can be taken as an index for our pandas dataframe and `diagnosis` is the label (either **M: malignant** or **B: benign**).

Let's prepare the data set first of all by (i) cleaning it, (ii) separating label from features, and (iii) splitting it into train and test sets.

In [ ]:
# drop last column (extra column added by pd)
data_1 = data.drop(data.columns[-1], axis=1)
# set column id as dataframe index
data_2 = data_1.set_index(data['id']).drop(data_1.columns[0], axis=1)

# check
data_2.tail()

We do a bit more preparation by converting the categorical labels into 1 for **M** and -1 for **B**.

In [ ]:
# convert categorical labels to numbers
diag_map = {'M': 1.0, 'B': -1.0}
data_2['diagnosis'] = data_2['diagnosis'].map(diag_map)

# put labels and features in different dataframes
y = data_2.loc[:, 'diagnosis']
X = data_2.iloc[:, 1:]

# check
print(y.tail())
X.tail()

As with any data set that has features over different ranges, it's required to standardise the data before.

In [ ]:
## EDIT THIS FUNCTION
def standardise(X):
  mu = np.mean(X, 0)
  sigma = np.std(X, 0)
  X_std = 2 ## <-- EDIT THIS LINE
  return X_std

In [ ]:
X_std = standardise(X)

# check
X_std.tail()

In [ ]:
# insert 1 in every row for intercept b
X_std.insert(loc=len(X_std.columns), column='intercept', value=1)

# split into train and test set
# stacking data X and labels y into one matrix
data_split = np.hstack((X_std, y[:, np.newaxis]))

# shuffling the rows        
np.random.shuffle(data_split)

# we split train to test as 70:30
split_rate = 0.7
train, test = np.split(data_split, [int(split_rate*(data_split.shape[0]))])

X_train = train[:,:-1]
y_train = train[:, -1]

X_test = test[:,:-1]
y_test = test[:, -1]

y_train = y_train.astype(float)
y_test = y_test.astype(float)

## Linear SVM
We start with defining the hinge loss as
$$
\mathcal L (\boldsymbol w) = \frac{1}{2} \| \boldsymbol w \|^2 + \frac{\lambda}{n} \sum_{i=1}^n \max \bigg( 0, 1-y_i (\boldsymbol w \cdot x_i + b) \bigg) \, .
$$
where $\boldsymbol w$ is the vector of weights, $\lambda$ the regularisation parameter, and $b$ the intercept which is included in our `X` as an additional column of $1$'s.

In [ ]:
# EDIT THIS FUNCTION
def compute_cost(W, X, y, regul_strength=1e5):
  n = X.shape[0]
  distances = 1 - ...  ## <-- EDIT THIS LINE
  distances[distances < 0] = 0  # equivalent to max(0, distance)
  hinge = regul_strength * ...  ## <-- EDIT THIS LINE

  # calculate cost
  cost = 1 / 2 * np.dot(W, W) + hinge
  return cost

Next, we need the gradients of this cost function.

In [ ]:
# calculate gradient of cost
def calculate_cost_gradient(W, X_batch, y_batch, regul_strength=1e5):
  # if only one example is passed
  if type(y_batch) == np.float64:
      y_batch = np.asarray([y_batch])
      X_batch = np.asarray([X_batch])  # gives multidimensional array

  distance = 1 - (y_batch * np.dot(X_batch, W))
  dw = np.zeros(len(W))

  for ind, d in enumerate(distance):
      if max(0, d)==0:
          di = W
      else:
          di = W - (regul_strength * y_batch[ind] * X_batch[ind])
      dw += di

  dw = dw/len(y_batch)  # average
  return dw

Both of the two previous functions are then used in SGD to update the weights iteratively with a given learning rate $\alpha$. We also implement a stop criterion that ends the learning as soon as the cost function has not changed more than a manually determined percentage.

We know that the learning happens through updating the weights according to
$$
\boldsymbol w = \boldsymbol w - \alpha \frac{\partial \mathcal L}{\partial \boldsymbol w}
$$

where $\frac{\partial \mathcal L}{\partial \boldsymbol w}$ is the gradient of the hinge loss we have computed in the previous cell.

In [ ]:
# EDIT THIS FUNCTION
def sgd(X, y, max_iterations=2000, stop_criterion=0.01, learning_rate=1e-5, regul_strength=1e5, print_outcome=False):
  # initialise zero weights
  weights = np.zeros(X.shape[1])
  nth = 0
  # initialise starting cost as infinity
  prev_cost = np.inf
  
  # stochastic gradient descent
  for iteration in range(1, max_iterations):
      # shuffle to prevent repeating update cycles
      np.random.shuffle([X, y])
      for ind, x in enumerate(X):
          ascent = ... ## <-- EDIT THIS LINE
          weights = weights - (learning_rate * ascent)

      # convergence check on 2^n'th iteration
      if iteration==2**nth or iteration==max_iterations-1:
          # compute cost
          cost = ...  ## <-- EDIT THIS LINE
          if print_outcome:
            print("Iteration is: {}, Cost is: {}".format(iteration, cost))
          # stop criterion
          if abs(prev_cost - cost) < stop_criterion * prev_cost:
              return weights
          
          prev_cost = cost
          nth += 1
  
  return weights

Now, we can take these functions and train a linear SVM with our training data.

In [ ]:
# train the model
W = sgd(X_train, y_train, max_iterations=2000, stop_criterion=0.01, learning_rate=1e-3, regul_strength=1e3, print_outcome=True)
print("Training finished.")

To evaluate the mean accuracy in both train and test set, we write a small function called `score`.

In [ ]:
## EDIT THIS FUNCTION
def score(W, X, y):
  y_preds = np.array([])
  for i in range(X.shape[0]):
    y_pred = np.sign(np.dot(X[i], W))
    y_preds = np.append(y_preds, y_pred)
  
  return np.float(sum(4)) / float(len(y)) ## <-- EDIT THIS LINE

In [ ]:
print("Accuracy on train set: {}".format(score(W, X_train, y_train)))
print("Accuracy on test set: {}".format(score(W, X_test, y_test)))

#### Questions:
1. What are other evaluation metrices besides the accuracy? Implement them and assess the performance of our classification algorithm with them.
2. What makes other evaluation metrices more appropriate given our unbalanced data set _(we have more benign than malignant examples)_?
3. Try different learning rates, regularisation strengths and number of iterations independently. What can you observe? Can you achieve higher accuracies?
4. What is your understanding why have we used the hinge loss with this data set of 31 features? 
5. Can you think of other loss functions instead of the hinge loss? What is your intuition how they will perform compared to the hinge loss? You could try implementing one and compare the results. 

## *T*-fold cross validation

Now we repeat the same procedure as above but do not only have one train-test split, but multiple in a *T*-fold cross validation method.

In [ ]:
def cross_val_split(data, num_folds):
  fold_size = int(len(data) / num_folds)
  data_perm = np.random.permutation(data)
  folds = []
  for k in range(num_folds):
    folds.append(data_perm[k*fold_size:(k+1)*fold_size, :])

  return folds

In [ ]:
# evaluate
folds = cross_val_split(train, 5)

In [ ]:
## EDIT THIS FUNCTION
def cross_val_evaluate(data, num_folds):
  
  folds = cross_val_split(data, num_folds)

  train_scores = []
  val_scores = []

  for i in range(len(folds)):
    print('Fold', i+1)
    # define the training set
    train_set = np.delete(np.asarray(folds).reshape(len(folds), folds[0].shape[0], folds[0].shape[1]), i, axis=0)
    train_folds = train_set.reshape(len(train_set)*train_set[0].shape[0], train_set[0].shape[1])
    X_train = train_folds[:,  1]  ## <-- EDIT THIS LINE
    y_train = train_folds[:, -1]
    
    # define the validation set
    val_fold = folds[0]  ## <-- EDIT THIS LINE
    X_val = val_fold[:,  1]  ## <-- EDIT THIS LINE
    y_val = val_fold[:, -1]

    # train the model
    W = sgd(X_train, y_train, max_iterations=1025, stop_criterion=0.01, learning_rate=1e-3, regul_strength=1e3)
    print("Training finished.")

    # evaluate
    train_score = score(W, X_train, y_train)
    val_score = score(W, X_val, y_val)
    print("Accuracy on train set #{}: {}".format(i+1, train_score))
    print("Accuracy on validation set #{}: {}".format(i+1, val_score))

    train_scores.append(train_score)
    val_scores.append(val_score)

  return train_scores, val_scores

In [ ]:
train_scores, val_scores = cross_val_evaluate(train, 5)

Finally, let's compute the mean accuracy.

In [ ]:
print(np.mean(val_scores))